In [1]:
import requests
from typing import List, Dict, Literal
import json
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

## Recap: Memory layer

New functionality: reset and tool role

In [4]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    # A new method
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    # A new method
    def reset(self) -> None:
        self._messages = []

In [5]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [6]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}}]

In [7]:
memory.reset()

In [8]:
memory.get_messages()

[]

## Recap: Chat function

New parameter: tools

In [9]:
def chat_with_tools(user_question:str=None, 
                    memory:Memory=None, 
                    model:str="gpt-4o-mini", 
                    temperature=0.0, 
                    tools=None)-> str:
    messages = [{"role": "user", "content": user_question}]
    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()        
    
    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools=tools, # Providing available tools to the model
    )
    
    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls # If the model decides to call a function
    
    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
    
    return ai_message

In [10]:
chat_with_tools(
    "2 to the power of -5?",
    model="gpt-3.5-turbo",
)

'1/32'

## Tool Call 101

Define functions that the model can access

In [1]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [2]:
power(2, 3)

8

In [13]:
power(2, -5)

0.03125

Create the Schema to pass to the model

In [14]:
# For more information, access `https://json-schema.org/docs`
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentatiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

Ask the questions providing the tools

In [19]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [20]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}}]

In [21]:
ai_message = chat_with_tools(
    "2 to the power of -5?",
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

The model decides whether to call a tool or not

In [22]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_4khpeRFWIt0taVJcTSAoiDbm', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]}]

Execute the Python function associated with the tool

In [23]:
tool_call_id = memory.last_message()['tool_calls'][0].id
tool_call_id

'call_4khpeRFWIt0taVJcTSAoiDbm'

In [24]:
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
args

{'base': 2, 'exponent': -5}

In [25]:
result = power(args["base"], args["exponent"])
result

0.03125

Feed the model back with the result

In [26]:
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)

In [27]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_4khpeRFWIt0taVJcTSAoiDbm', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_4khpeRFWIt0taVJcTSAoiDbm'}]

In [28]:
ai_message = chat_with_tools(
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [29]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_4khpeRFWIt0taVJcTSAoiDbm', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_4khpeRFWIt0taVJcTSAoiDbm'},
 {'role': 'assistant',
  'content': '2 to the power of -5 is 0.03125.',
  'tool_calls': None}]

In [22]:
def random_got_quote():
    response = requests.get("https://api.gameofthronesquotes.xyz/v1/random")
    return str(response.json())

In [ ]:
def get_audio_files(company_name:str):
    audio_files_map = {
        "apple": "apple.wav",
        "microsoft": "microsoft.wav",
        "google": "alphabet.wav",
        "amazon": "amazon.wav",
        "facebook": "meta.wav",
    }
    return audio_files_map[company_name]

In [ ]:
def transcribe_audio_file(audio_file:str):
    earnings_calls_2025 = {
        "apple.wav": "Apple reported strong Q1 2025 earnings, driven by record iPhone 16 sales and increased revenue from its subscription services. The company also announced advancements in AI-powered features for iOS and continued investment in augmented reality technologies.",
        "microsoft.wav": "Microsoft's revenue surged in 2025, fueled by Azure’s dominance in cloud computing and the expansion of AI-powered productivity tools in Office 365. The company also highlighted strong growth in gaming, with record engagement on Xbox and Game Pass.",
        "alphabet.wav": "Alphabet reported steady revenue growth, with Google Search and YouTube ad revenue rebounding after economic slowdowns. Google Cloud continues to gain enterprise customers, while the company’s AI models saw widespread adoption across industries.",
        "amazon.wav": "Amazon posted record-breaking earnings, with AWS leading cloud market growth. E-commerce sales remained strong, boosted by AI-driven logistics optimizations. The company also emphasized expansions in drone delivery and AI-powered Alexa services.",
        "meta.wav": "Meta’s earnings exceeded expectations, with continued revenue growth from Instagram and WhatsApp. The company doubled down on AI-powered content discovery and virtual reality adoption in the metaverse, with Quest headsets driving user engagement."
    }
    
